In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("yolov9_paper.pdf") 
data = loader.load()  # entire PDF is loaded as a single Document
# data

# we can also take the data from website just we have to pass the website link by this we can create a company chatbot

In [4]:
len(data)

18

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  96


In [6]:
docs[7]

Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-03-01T01:40:26+00:00', 'author': '', 'keywords': '', 'moddate': '2024-03-01T01:40:26+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'yolov9_paper.pdf', 'total_pages': 18, 'page': 1, 'page_label': '2'}, page_content='ditional layers to combine repeatedly fed input data, which\nwill significantly increase the inference cost. In addition,\nsince the input data layer to the output layer cannot have a\ntoo deep path, this limitation will make it difficult to model\nhigh-order semantic information during the training pro-\ncess. As for masked modeling, its reconstruction loss some-\ntimes conflicts with the target loss. In addition, most mask\nmechanisms also produce incorrect associations with data.\nFor the deep supervision mechanism, it will produce error\nac

In [7]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
load_dotenv() 

#Get an API key: 
# Head to https://ai.google.dev/gemini-api/docs/api-key to generate a Google AI API key. Paste in .env file

# Embedding models: https://python.langchain.com/v0.1/docs/integrations/text_embedding/

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]
#vector

[0.05636945366859436,
 0.004828543867915869,
 -0.07625909894704819,
 -0.023642510175704956,
 0.053293220698833466]

In [8]:
vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))


In [9]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

retrieved_docs = retriever.invoke("What is new in yolov9?")

In [10]:
len(retrieved_docs)


10

In [11]:
print(retrieved_docs[5].page_content)


RT DETR-R18 [43] (I) 20 60 46.5 63.8 – – – –
YOLOv9-M (S) 20.0 76.3 51.4 68.1 56.1 33.6 57.0 68.0
Gold YOLO-S [61] (C) 21.5 46.0 46.4 63.4 – – – –
Gold YOLO-S [61] (D) 21.5 46.0 46.1 63.3 – – – –
Gold YOLO-S [61] (I) 21.5 46.0 45.5 62.2 – – – –
PPYOLOE+-M [74] (C) 23.4 49.9 49.8 67.1 54.5 31.8 53.9 66.2
PPYOLOE-M [74] (I) 23.4 49.9 49.0 66.5 53.0 28.6 52.9 63.8
RTMDet-M [44] (I) 24.7 78.6 49.4 66.8 – – – –
YOLOv9-C (S) 25.3 102.1 53.0 70.2 57.8 36.2 58.5 69.3
DAMO YOLO-M [75] (D) 28.2 61.8 50.4 67.2 55.1 31.6 55.3 67.1
RT DETR-R34 [43] (I) 31 92 48.9 66.8 – – – –
RT DETR-L [43] (I) 32 110 53.0 71.6 57.3 34.6 57.3 71.2
YOLOv9-E (S) 34.7 147.1 54.5 71.7 59.2 38.1 59.9 70.3
YOLOv6-M v3.0 [30] (D) 34.9 85.8 50.0 66.9 – – – –
RT DETR-R50M [43] (I) 36 100 51.3 69.6 – – – –
Gold YOLO-M [61] (C) 41.3 57.5 51.1 68.5 – – – –
Gold YOLO-M [61] (D) 41.3 57.5 50.9 68.2 – – – –
Gold YOLO-M [61] (I) 41.3 57.5 50.2 67.5 – – – –
RT DETR-R50 [43] (I) 42 136 53.1 71.3 57.7 34.8 58.0 70.0


In [17]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",temperature=0.3, max_tokens=500)

In [18]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [19]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [20]:
response = rag_chain.invoke({"input": "what is new in YOLOv9?"})
print(response["answer"])

YOLOv9 introduces two main innovations: Programmable Gradient Information (PGI) and Generalized ELAN (GELAN). The network architecture replaces ELAN with GELAN, utilizing CSPNet blocks and RepConv as computational blocks. Additionally, YOLOv9 features a simplified downsampling module and an optimized anchor-free prediction head.
